# Exercise 03 : Run MLflow Project

MLflow project is a portable and reusable packaging formats for ML code and deliverables.

In this exercise, we'll run MLflow projects on Azure Machine Learning compute (remote compute).

*back to [index](https://github.com/tsmatz/mlflow-azureml/)*

## 1. Create Azure ML compute (remote compute)

Connect to Azure Machine Learning workspace.

In [1]:
from azureml.core import Workspace

ws = Workspace.get(
    name = "<FILL-AML-WORKSPACE-NAME>",
    subscription_id = "<FILL-AZURE-SUBSCRIPTION-ID>",
    resource_group = "<FILL-RESOUCE-GROUP-NAME>")

Create Azure ML compute (remote compute).<br>
By enabling auto-scaling (from 0 to 1), the node will be terminated if it's inactive.<br>
(If VM already exists, this script will pick up the existing one.)

In [2]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name="myvm01")
    print("found existing:", compute_target.name)
except ComputeTargetException:
    print("creating new.")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size="Standard_DS2_v2",
        min_nodes=0,
        max_nodes=1)
    compute_target = ComputeTarget.create(ws, "myvm01", compute_config)
    compute_target.wait_for_completion(show_output=True)

creating new.
InProgress.
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## 2. Run local MLflow project

Set AML workspace for MLflow backend service.

In [3]:
import mlflow

tracking_uri = ws.get_mlflow_tracking_uri()
mlflow.set_tracking_uri(tracking_uri)

Create a project folder, if not exists.

In [4]:
import os
script_folder = "./exercise03-test-project"
os.makedirs(script_folder, exist_ok=True)

Create entry script (training script) named ```train.py```.<br>
This is almost same as "[Exercise 01 : Track logs and metrics](./01_track_logs.ipynb)".

In [5]:
%%writefile exercise03-test-project/train.py
import os
import sys
import argparse

import pandas as pd
from sklearn.linear_model import ElasticNet
import mlflow

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        '--alpha',
        type=float,
        default='0.5')
    parser.add_argument(
        '--l1_ratio',
        type=float,
        default='0.1')
    FLAGS, unparsed = parser.parse_known_args()

    # Read the wine-quality csv file from the URL
    csv_url = (
        "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
    )
    data = pd.read_csv(csv_url, sep=";")

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = data.drop(["quality"], axis=1)
    train_y = data[["quality"]]

    mlflow.sklearn.autolog()

    with mlflow.start_run() as my_run:
        lr = ElasticNet(alpha=FLAGS.alpha, l1_ratio=FLAGS.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

    print("completed training with alpha={} l1_ratio={}".format(FLAGS.alpha, FLAGS.l1_ratio))

Writing exercise03-test-project/train.py


Create ```MLproject``` file, which is a text project's definition written in YAML.

In [6]:
%%writefile exercise03-test-project/MLproject
name: exercise03_local_project

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py --alpha {alpha} --l1_ratio {l1_ratio}"

Writing exercise03-test-project/MLproject


Create conda file ```conda.yaml```.

In [7]:
%%writefile exercise03-test-project/conda.yaml
name: tutorial
channels:
  - defaults
  - anaconda
  - conda-forge
dependencies:
  - numpy
  - pandas
  - scikit-learn
  - pip
  - pip:
    - mlflow
    - azureml-mlflow

Writing exercise03-test-project/conda.yaml


Now let's run MLflow project on AML remote compute ```myvm01```. (This compute is created by above script.)<br>
This will take a while, since it will run the following task.

- Create a docker image and register in Azure container registry (ACR).
- Scale up compute (```myvm01```).
- Create conda environment.
- Run script.

In [8]:
# Set experiment name
experiment_name = "exercise03-local-mlproject"
mlflow.set_experiment(experiment_name)
# Run MLflow project
run1 = mlflow.projects.run(
    uri="./exercise03-test-project",
    parameters={"alpha":0.3},
    backend = "azureml",
    backend_config = {"COMPUTE": "myvm01", "USE_CONDA": True},
    synchronous=True)

2022/03/10 05:57:37 INFO mlflow.tracking.fluent: Experiment with name 'exercise03-local-mlproject' does not exist. Creating a new experiment.
Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022/03/10 05:57:37 INFO mlflow.projects.utils: === Created directory /tmp/tmpavbovj_n for downloading remote URIs passed to arguments of type 'path' ===
No Python version provided, defaulting to "3.6.2"
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
Class AzureMLSubmittedRun: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RunId: exercise03-local-mlproject_1646891859_f2732d93
Web View: https://ml.azure.com/runs/exercise03-local-mlproject_1646891859_f2732d93?wsid=/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/AzureML-rg/workspaces/ws01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Streaming azureml-logs/20_image_build_log.txt

2022/03/10 05:57:51 Downloading source code...
2022/03/10 05:57:52 Finished downloading source code
2022/03/10 05:57:52 Creating Docker network: acb_default_network, driver: 'bridge'
2022/03/10 05:57:52 Successfully set up Docker network: acb_default_network
2022/03/10 05:57:52 Setting up Docker configuration...
2022/03/10 05:57:53 Successfully set up Docker configuration
2022/03/10 05:57:53 Logging in to registry: 40c749df66a2408086e625acecf8e129.azurecr.io
2022/03/10 05:57:54 Successfully logged into 40c749df66a2408086e625acecf8e129.azurecr.io
2022/03/10 05:57:54 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network

Ran pip subprocess with arguments:
['/azureml-envs/azureml_fe56adca9a0c3e8e98ff52a884269349/bin/python', '-m', 'pip', 'install', '-U', '-r', '/azureml-environment-setup/condaenv.2wpx1bzh.requirements.txt']
Pip subprocess output:
  Created wheel for databricks-cli: filename=databricks_cli-0.16.4-py3-none-any.whl size=106877 sha256=298ec31a08219f4911fc8c21cee17d8e896cc8e628f662d2ac420801d5f9089a
  Stored in directory: /root/.cache/pip/wheels/44/08/b0/8d0a22c4625004e1b16012ecfc1efcb515d54ba674dd94f29e
Successfully built databricks-cli

done
#
# To activate this environment, use
#
#     $ conda activate /azureml-envs/azureml_fe56adca9a0c3e8e98ff52a884269349
#
# To deactivate an active environment, use
#
#     $ conda deactivate



==> WARNING: A newer version of conda exists. <==
  current version: 4.9.2
  latest version: 4.11.0

Please update conda by running

    $ conda update -n base -c defaults conda


Removing intermediate container 1e831987907f
 ---> a60b9d828817
Step 9/21 : ENV PAT

 ---> Running in 7cdec2d8c164
Removing intermediate container 7cdec2d8c164
 ---> 8d621f7b90d4
Step 17/21 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> 661d0f43145c
Step 18/21 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in 3f5400d71256
Removing intermediate container 3f5400d71256
 ---> f73304cc2281
Step 19/21 : RUN rm -rf azureml-environment-setup
 ---> Running in dc3e9d59f963
Removing intermediate container dc3e9d59f963
 ---> 53d2defc6ea6
Step 20/21 : ENV AZUREML_ENVIRONMENT_IMAGE True
 ---> Running in a16c28aef068
Removing intermediate container a16c28aef068
 ---> 49ff0013ba63
Step 21/21 : CMD ["bash"]
 ---> Running in 61b607762613
Removing intermediate container 61b607762613
 ---> 7146b960f1cd
Successfully built 7146b960f1cd
Successfully tagged 40c749df66a2408086e625acecf8e129.azurecr.io/azureml/azureml_3737395f9ad4

2022/03/10 06:14:05 INFO mlflow.projects: === Run (ID 'exercise03-local-mlproject_1646891859_f2732d93') succeeded ===


## 3. Run MLflow project in GitHub

You can also register your MLflow project in GitHub repository, and run this repository by MLflow API.

1) First, login and create a repository in [GitHub](https://github.com/).

2) Run the following command in console, and register local repository ```exercise03-test-project``` into GitHub.<br>
(**Replace ```<ACCOUNT-NAME>``` and ```<REPOSITORY-NAME>```.**)

```
cd exercise03-test-project
git init
git add .
git commit -m "first commit"
git remote add origin https://github.com/<ACCOUNT-NAME>/<REPOSITORY-NAME>.git
git push -u origin master
cd ..
```

7) Run remote MLflow project in GitHub on Azure ML compute.<br>
(**Replace ```<ACCOUNT-NAME>``` and ```<REPOSITORY-NAME>```.**)

In [10]:
# Set experiment name
experiment_name = "exercise03-github-mlproject-test01"
mlflow.set_experiment(experiment_name)
# Run MLflow project
run1 = mlflow.projects.run(
    uri="https://github.com/<ACCOUNT-NAME>/<REPOSITORY-NAME>",
    parameters={"alpha":0.3},
    backend = "azureml",
    backend_config = {"COMPUTE": "myvm01", "USE_CONDA": True},
    synchronous=True)

Class AzureMLProjectBackend: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
2022/03/10 06:47:58 INFO mlflow.projects.utils: === Fetching project from https://github.com/tsmatsuz/mlflow-test-proj into /tmp/tmp5a7nhher ===
2022/03/10 06:47:58 INFO mlflow.projects.utils: === Created directory /tmp/tmp37jzukm8 for downloading remote URIs passed to arguments of type 'path' ===
No Python version provided, defaulting to "3.6.2"
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.
Class AzureMLSubmittedRun: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


RunId: exercise03-github-mlproject-test01_1646894881_c8269d2e
Web View: https://ml.azure.com/runs/exercise03-github-mlproject-test01_1646894881_c8269d2e?wsid=/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/AzureML-rg/workspaces/ws01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47

Execution Summary
RunId: exercise03-github-mlproject-test01_1646894881_c8269d2e
Web View: https://ml.azure.com/runs/exercise03-github-mlproject-test01_1646894881_c8269d2e?wsid=/subscriptions/b3ae1c15-4fef-4362-8c3a-5d804cdeb18d/resourcegroups/AzureML-rg/workspaces/ws01&tid=72f988bf-86f1-41af-91ab-2d7cd011db47



2022/03/10 06:54:18 INFO mlflow.projects: === Run (ID 'exercise03-github-mlproject-test01_1646894881_c8269d2e') succeeded ===
